##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reuse pre-existing TF1.x checkpoints


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/migrate/validate_correctness"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/migrate/validate_correctness.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/migrate/validate_correctness.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/migrate/validate_correctness.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

As seen in the above testing, `get_variable` calls in decorators for `tf.Module` and `tf.keras.layers.Layer` methods end up maintaining the variable naming semantics of `get_variable` in TF1.x graphs and sessions.

Because [`tf.train.Checkpoint`](https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint) supports loading legacy `tf.compat.v1.train.Saver`-generated name-based checkpoints and your variable names are likely to match, you can use your pre-existing TF1.x name-based checkpoints often entirely out of the box.

## Setup

## Loading checkpoints with no assignment mapping

If your TF1.x program loaded checkpoints directly with no assignment mapping to map from saved to loaded variable names (such as when warm-starting), and your variable names from `tf.keras.layers.Layer` successfully matched the variable names exactly in the above tests, then you can just load your existing checkpoint directly with `tf.train.Checkpoint`.

The `tf.train.Checkpoint` [guide](https://www.tensorflow.org/guide/checkpoint) describes APIs which you can use to validate that any and all variables you expect to have been loaded are actually loaded. Remember that `get_variable` weights will only get created when your model's forward pass runs.

After loading the checkpoint and running it, consider saving out a new `tf.train.Checkpoint` that uses the TF2 object-oriented style and reusing that one. You should save out the TF2 object-oriented style checkpoint once your variables have been created.

## Loading checkpoints with a simple nested variable_scope assignment map

TF1.x code that used checkpoints for warm-starting just the backbone of a model often relies on an *assignment map*, such as the `assignment_map` argument of `tf.compat.v1.train.init_from_checkpoint`.

If the assignment map rule is a simple one like `{"/": "backbone_name/"}` (which tries to map any variable named "FOO" in your checkpoint to the corresponding variable named "backbone_name/FOO" in your model), then it may be possible to either add or remove a `variable_scope` in your `track_tf1_style_variable`-decorated call method to make sure the variable names exactly match the names in your checkpoint with no need for an assignment map.

Note: As described in past sections you still need to nest `tf.compat.v1.layers` that rely on autogenerated names inside at least one `variable_scope` to ensure variables get re-used correctly.

You can then load your existing checkpoint as is with `tf.train.Checkpoint`, and save out a new object-oriented checkpoint right away.

## Loading checkpoints with a more complicated assignment mapping

If adjusting the `variable_scope` nesting in your shim-decorated call method proves insufficient, or you have a more complicated name and scope assignment mapping, you may need to directly rename the variables in your saved TF1.x name-based checkpoint to the expected variable names post-mapping. There are several approaches to do this like using a dictionary mapping that contains all variable names, using regular expressions to rewrite any scopes, etc. The following code snippets demonstrate a few of these methods.

This first code snippet shows how you can programmatically grab the names from an existing name-based checkpoint.
```
CKPT = "/PATH/TO/ORIGINAL/CHECKPOINT"
with tf.Graph().as_default():
 checkpoint_reader = tf.train.NewCheckpointReader(CKPT)
 names_to_shapes = checkpoint_reader.get_variable_to_shape_map()
 var_names = sorted(list(names_to_shapes))
 for name in var_names:
   print(name)
```

This second code snippet shows how you can rename variables in one TF1.x name-based checkpoint and save out a new name-based checkpoint (if you have a dict `new_name_from_old_name` mapping old variable names to post-assignment-mapping variable names).
```
to_migrate = [
    ("/PATH/TO/ORIGINAL/CHECKPOINT",
     "/PATH/TO/CHECKPOINT/WITH/RENAMED/VARIABLES"),
]
for input_ckpt, output_ckpt in to_migrate:
  print(f'Migrating {input_ckpt} to {output_ckpt}')
  with tf.Graph().as_default():
    checkpoint_reader = tf.train.NewCheckpointReader(input_ckpt)
    variable_from_new_name = {}
  
    for old_name, new_name in new_name_from_old_name.items():
      value = checkpoint_reader.get_tensor(old_name)
      variable_from_new_name[new_name] = tf.Variable(value)
  
    init = tf.global_variables_initializer()
    saver = tf.train.Saver(variable_from_new_name)
  
    with tf.Session() as sess:
      sess.run(init)
      saver.save(sess, output_ckpt, write_meta_graph=False)

  for old_name, new_name in new_name_from_old_name.items():
    print(f'{old_name} -> {new_name}')
```

Once you have remapped the names in your variable name checkpoint according to the assignment map, you should then be able to load it and afterwards save out a new object-oriented checkpoint.

## Loading checkpoints when not all variables are created by `get_variable`

If your shim-decorated layer or module consists of some variables (or Keras layers/models) that use `tf.Variable` instead of `tf.compat.v1.get_variable` and get attached as properties/tracked in an object oriented way, they may have different variable naming semantics in TF1.x graphs/sessions versus during eager exection.

So, you will have to compare these variable names before and after and explicitly rewrite the checkpoint with renamed variables, just as for more complex assignment mapping above.

Once you do that you can load the checkpoint, and save out a new object-oriented checkpoint for future use.

Warning: Variables may have duplicate names in eager execution, which may cause problems if multiple variables in the name-based checkpoint need to be mapped to the same name. You may be able to explicitly adjust the layer and variable names using `tf.name_scope` and layer constructor or `tf.Variable` `name` arguments to adjust variable names and ensure there are no duplicates.

## Resources

Refer to the following guides:
* Validating numerical equivalence and correctness
* TF Modeling Shims